# hw9
使用iris数据集进行神经网络练习：其中variety是标签，其他四个是输入特征。构建神经网络（架构不限）对variety
进行预测，输入为sepal.length，sepal.width，petal.length，petal.width四个变量。可对数据集分为训练集和测试集，
比较模型在训练集和测试集上的预测效果。

In [34]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [35]:
iris = pd.read_csv("iris.csv")
iris.head()

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa
3,4.6,3.1,1.5,0.2,Setosa
4,5.0,3.6,1.4,0.2,Setosa


In [36]:
X = iris[["sepal.length", "sepal.width", "petal.length", "petal.width"]].values
y = iris["variety"].values
y = LabelEncoder().fit_transform(y)

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)

In [47]:
class IrisNN(nn.Module):
    def __init__(self) -> None:
        super(IrisNN, self).__init__()
        self.fc1 = nn.Linear(4, 10)
        self.fc2 = nn.Linear(10, 5)
        self.fc3 = nn.Linear(5, 3)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [48]:
iris_model = IrisNN()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(iris_model.parameters(), lr=0.01)
num_epochs = 50

In [49]:
for epoch in range(num_epochs):
    iris_model.train()
    optimizer.zero_grad()
    outputs = iris_model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()
    if (epoch + 1) % 5 == 0:
        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}")

Epoch [5/50], Loss: 1.0645
Epoch [10/50], Loss: 1.0248
Epoch [15/50], Loss: 0.9557
Epoch [20/50], Loss: 0.8521
Epoch [25/50], Loss: 0.7405
Epoch [30/50], Loss: 0.6326
Epoch [35/50], Loss: 0.5521
Epoch [40/50], Loss: 0.4966
Epoch [45/50], Loss: 0.4537
Epoch [50/50], Loss: 0.4130


In [50]:
with torch.no_grad():
    outputs_test = iris_model(X_test)
    _, predicted_test = torch.max(outputs_test, 1)
    accuracy_test = (predicted_test == y_test).sum().item() / y_test.size(0)
    print(f"Test Accuracy: {accuracy_test * 100:.3f}%")

Test Accuracy: 96.667%
